In [ ]:
このColabノートブックは、https://github.com/compphoto/BoostingMonocularDepth のグローバルブースティングメソッドを適用するのを支援します。これにより、ダブル推定を生成するために使用するマージングネットワークを単一のパスで実行します。

手順：

「ランタイム/ランタイムのタイプを変更」をGPUに設定して、GPUを使用していることを確認します。
コードセクション1を実行します。
低解像度のベースデプスを「/content/BoostYourOwnDepth/input/low-res」に配置します。
（画像のR20解像度を計算したい場合は、以下の手順を実行できます: a.) RGB画像を「/content/BoostYourOwnDepth/input/rgb」に配置します。 b.) セクション2を実行して、与えられた画像に対して最適な入力解像度を取得します。 c.) b.) で得られた解像度を入力サイズとしてネットワークからデプス推定を作成します。）
高解像度のデプスを「/content/BoostYourOwnDepth/input/high-res」に配置します。
コードセクション3を実行します。
結果は「src/outputs」に生成されます。

In [ ]:
Section 1

セクション1では、リポジトリをクローンし、マージングネットワークの重みをダウンロードしています。

In [2]:
# ライブラリをインストール
!pip install pillow matplotlib numpy torchvision opencv-contrib-python scikit-image torch gdown

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [10]:
# Code section 1

# Clone repo
!git clone https://github.com/compphoto/BoostYourOwnDepth.git

# Downloading merge model weights
!mkdir -p /app/src/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/
# これダウンロードされない場合はここから持ってこれます https://www.sfu.ca/~yagiz/CVPR21/
!wget -c https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth -O /app/src/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/latest_net_G.pth
!gdown https://drive.google.com/u/0/uc?id=1cU2y-kMbt0Sf00Ns4CN2oO9qPJ8BensP&export=download

!ls /app/src/BoostYourOwnDepth/pix2pix/checkpoints/mergemodel/


fatal: destination path 'BoostYourOwnDepth' already exists and is not an empty directory.
--2023-06-26 01:11:36--  https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth
sfu.ca (sfu.ca) をDNSに問いあわせています... 142.58.103.137, 142.58.103.55, 142.58.228.150, ...
sfu.ca (sfu.ca)|142.58.103.137|:443 に接続しています... 接続しました。
SSL による接続が確立できません。
latest_net_G.pth


In [ ]:
Section 2

次のセクションを実行して、選択した画像のR20解像度を計算してください。重要な点：これは実際のデプス推定を行いません。

In [11]:
%cd /app/src/

# Code section 2
# np.float) → np.float.64)
from BoostYourOwnDepth.utils import calculateprocessingres, read_image
import os

############################################################################
#### Set the image name that you want to optimize for, must be present in /content/BoostYourOwnDepth/input/rgb ######################
image_name = 'test_01.jpg'
############################################################################

image_path = os.path.join('/app/src/BoostYourOwnDepth/input/rgb',image_name)
img = read_image(image_path)

# Hyperparameters
whole_size_threshold = 3000  # R_max from the paper
r_threshold_value = 0.2 # Value x of R_x defined in the section 5 of the main paper.
scale_threshold = 3  # Allows up-scaling with a scale up to 3

##########################################################################
##### Fill in the receptive field size of your network here ##############
net_receptive_field_size = 100
##########################################################################

# Find the best input resolution R-x. The resolution search described in section 5-double estimation of the main paper and section B of the
# supplementary material.
whole_image_optimal_size, _ = calculateprocessingres(img, net_receptive_field_size,r_threshold_value, scale_threshold, whole_size_threshold)

/app/src


In [ ]:
Section 3

「/app/src/BoostYourOwnDepth/input/」内の対応するフォルダに低解像度のデプスマップと高解像度のデプスマップを配置してください。その後、次のセクションを実行してマージされた推定を生成することができます。このセクションでは、入力からダブル推定を作成するために、マージングネットワークを単一のパスで実行します。重要な点：読み取りエラーを回避するために、高解像度と低解像度のファイル名が同じであることを確認してください。

In [10]:
# Code section 3
!python3 /app/src/BoostYourOwnDepth/boost_depth.py --data_dir /app/src/BoostYourOwnDepth/input/ --output_dir /app/src/BoostYourOwnDepth/output --checkpoints_dir /app/src/BoostYourOwnDepth/pix2pix/checkpoints --colorize_results

device: cuda
Namespace(data_dir='/app/src/BoostYourOwnDepth/input/', output_dir='/app/src/BoostYourOwnDepth/output', checkpoints_dir='/app/src/BoostYourOwnDepth/pix2pix/checkpoints', output_resolution=1, pix2pixsize=1024, colorize_results=True, max_res=inf)
----------------- Options ---------------
                    Final: False                         
                       R0: False                         
                      R20: False                         
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /app/src/BoostYourOwnDepth/pix2pix/checkpoints	[default: ./pix2pix/checkpoints]
         colorize_results: True                          	[default: False]
                crop_size: 672                           
                 data_dir: /app/src/BoostYourOwnDepth/input/	[default: None]
                 dataroot: None                          
             dataset_mode: depthmer

In [ ]:
自由に実験してみてください！
たとえば、入力のデプスマップを編集したり、異なるデプス推定器を組み合わせたり、独自のモデルを使用したりすることができます。